# Importing the necessary libraries

In [ ]:
import yaml
import os
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
import json

# Transforming the drivers table:

## getting all possible keys in all yaml files

In [ ]:
folder_path = "/home/floppabox/f1/f1db/src/data/drivers"
max_length = 0
keys = []
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        data=yaml.safe_load(file)
        if max_length<=len(list(data.keys())):
            max_length = len(list(data.keys()))
            keys = list(data.keys())

print(f'there are {max_length} keys in drivers:')
print(keys)


In [ ]:
spark = SparkSession.builder.appName("YAML to CSV").getOrCreate()


In [ ]:
drivers_data, drivers_relationships_data = [], []

# Extracting all drivers data

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        record ={}

        for key in keys:
            if key != 'familyRelationships':
                record[key]= content_drivers.get(key)

        print(record)
        drivers_data.append(record)


In [ ]:

id=0
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        
        if 'familyRelationships' in content_drivers.keys():
            for rel in content_drivers['familyRelationships']:
                record ={}
                record['id']=id
                id=id+1
                record['driverId']=content_drivers.get('id')
                record['relationId']=rel.get('driverId')
                record['type']=rel.get('type')
                drivers_relationships_data.append(record)




In [ ]:
real_keys= [x for x in keys if x != 'familyRelationships']

drivers= spark.createDataFrame(drivers_data).select(real_keys)

drivers_relationships = spark.createDataFrame(drivers_relationships_data).select(['id','driverId', 'relationId', 'type'])

drivers_relationships.show()